# Dilution time to equilibrium

This notebook performs analyses described in the 

**Manuscript:**

Modelling the decamerisation cycle of PRDX1 and the inhibition-like effect on its peroxidase activity

**Authors:**

C. Barry, C. Pillay, J. Rohwer

**Purpose:**

Performs simulations of dilutions of Prx solutions

Plot: Figure 8a, 8b, S7

**Requirements:**

Python libraries (see **Imports** below)

Prx parameters (filename: `fitted_koff_kon_PRDX1_params.csv`)

Prx dimer-decamer dilution model (`Prx_dim-dec_dil.psc`)

##### Imports

In [ ]:
import os
import csv
import math
import copy

import scipy as sp
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import pysces


###### Get directory paths

In [ ]:
prev_dir = os.path.split(os.getcwd())[0]

mod_dir = os.path.join(prev_dir,"models")
if not os.path.isdir(mod_dir): os.mkdir(mod_dir) # ensure dir exists

par_dir = os.path.join(prev_dir,"params")
if not os.path.isdir(par_dir): os.mkdir(par_dir) # ensure dir exists
    
fig_dir = os.path.join(os.getcwd(),"Figures")
if not os.path.isdir(fig_dir): os.mkdir(fig_dir) # ensure dir exists


###### Matplotlib stuff

In [ ]:
%matplotlib inline

mpl_width = 4.5
mpl_height = 3.5
mpl_dpi = 600
mpl_xlabel_fontsize = "large"
mpl_ylabel_fontsize = "large"
mu = "\u03bc"


##### params

In [ ]:
decamer_param_file = "fitted_koff_kon_PRDX1_params.csv"
onestep_pardict = pd.read_csv(os.path.join(par_dir,decamer_param_file),
                              index_col=0).squeeze('columns').to_dict()


## dilution assay functions

In [ ]:
def do_dilution_assay(mod1,
                    dilution_factor,
                    vary="SH_SH_init",
                    vary_init= False,
                    end_time=0.1,
                    points=1000,
                    y_data_out='SH_SH__SH_SH__SH_SH__SH_SH__SH_SH'):
    
    """
    Performs a simulation of diluting a Prx protein solution. 
    
    Outputs the mol fraction of Prx and time array
    """
    # Copy model
    mod = copy.deepcopy(mod1)
    mod.SetQuiet()
    
    # Set expicit experimental concentration
    if vary_init == False:
        vary_init = getattr(mod,vary)
    setattr(mod,vary,vary_init)
    
    # Calculate steady state and copy values as initial for simulation
    mod.doState()
    for i in range(len(mod.species)):
        setattr(mod, f"{mod.species[i]}_init", mod.state_species[i]*1/dilution_factor)
    
    # Do sim
    mod.sim_end = end_time
    mod.sim_points = points
    mod.Simulate()
    
    # Grab data
    x_data = mod.sim["Time"]
    
    t_Prx_dimers = (mod.sim["SH_SH"] + mod.sim["SH_SH__SH_SH__SH_SH__SH_SH__SH_SH"]*5)
    
    y_data_num_multiplier = len(y_data_out.split("__"))
    y_data_num = mod.sim[y_data_out]*y_data_num_multiplier
        
    y_data_fmol = y_data_num/t_Prx_dimers
    y_data = y_data_num
    
    return x_data, y_data_fmol, y_data


## dilution assays

In [ ]:
dilution_range = [1,2,5,10,50,100]


In [ ]:
# Load model
mod_Prx_dil = pysces.model("Prx_dim-dec_dil.psc", dir=mod_dir)
mod_Prx_dil.mode_integrator = 'CVODE'
mod_Prx_dil.__dict__.update(onestep_pardict)  # this updates the model in a single step
mod_Prx_dil.k_Prx_asso = mod_Prx_dil.kon
mod_Prx_dil.k_Prx_dec_disasso = mod_Prx_dil.koff
mod_Prx_dil.showModel()

### 450 μM Prx

In [ ]:
# Do sim
dim_init = 450
result_dil_450um_Prx = list(map(lambda dil_factor: do_dilution_assay(mod_Prx_dil, 
                                                                     dil_factor, 
                                                                     vary_init=dim_init), 
                                dilution_range))


In [ ]:
# Plot results
cols = 1
rows = 1

f, axarr = plt.subplots(rows, cols)
f.set_size_inches(w=mpl_width, h=mpl_height)

for count,result in enumerate(result_dil_450um_Prx):
    axarr.plot(result[0],result[1],label=f"{dilution_range[count]}x")

axarr.set_xlabel("Time (sec)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("$f_{mol}$ Prx decamers",fontsize=mpl_ylabel_fontsize)
axarr.set_ylim(0,1)

axarr.legend(title="Dilution",loc="lower right")
f.tight_layout()

### 100 μM Prx (figure S7a)

In [ ]:
# Do sim
dim_init = 100
result_dil_100um_Prx = list(map(lambda dil_factor: do_dilution_assay(mod_Prx_dil, 
                                                                     dil_factor, 
                                                                     vary_init=dim_init),
                                dilution_range))


In [ ]:
# Plot results
cols = 1
rows = 1

f, axarr = plt.subplots(rows, cols)
f.set_size_inches(w=mpl_width, h=mpl_height)

for count,result in enumerate(result_dil_100um_Prx):
    axarr.plot(result[0],result[1],label=f"{dilution_range[count]}x")

axarr.set_xlabel("Time (sec)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("$f_{mol}$ Prx decamers",fontsize=mpl_ylabel_fontsize)
axarr.set_ylim(0,1)

axarr.legend(title="Dilution",loc="lower right")
f.tight_layout()
f.savefig(os.path.join(fig_dir,"dil_fmol_Prx_decamer_vs_time_100um.pdf"),dpi= mpl_dpi)

### 10 μM Prx (figure S7b)

In [ ]:
# Do sim
dim_init = 10
result_dil_10um_Prx = list(map(lambda dil_factor: do_dilution_assay(mod_Prx_dil, 
                                                                    dil_factor, 
                                                                    vary_init=dim_init), 
                               dilution_range))


In [ ]:
# Plot results
cols = 1
rows = 1

f, axarr = plt.subplots(rows, cols)
f.set_size_inches(w=mpl_width, h=mpl_height)

for count,result in enumerate(result_dil_10um_Prx):
    axarr.plot(result[0],result[1],label=f"{dilution_range[count]}x")

axarr.set_xlabel("Time (sec)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("$f_{mol}$ Prx decamers",fontsize=mpl_ylabel_fontsize)
axarr.set_ylim(0,1)

axarr.legend(title="Dilution")
f.tight_layout()
f.savefig(os.path.join(fig_dir,"dil_fmol_Prx_decamer_vs_time_10um.pdf"),dpi= mpl_dpi)

### 1 μM Prx (figure S7c)

In [ ]:
# Do sim
dim_init = 1
result_dil_1um_Prx = list(map(lambda dil_factor: do_dilution_assay(mod_Prx_dil, 
                                                                   dil_factor, 
                                                                   vary_init=dim_init), 
                              dilution_range))


In [ ]:
# Plot results
cols = 1
rows = 1

f, axarr = plt.subplots(rows, cols)
f.set_size_inches(w=mpl_width, h=mpl_height)

for count,result in enumerate(result_dil_1um_Prx):
    axarr.plot(result[0],result[1],label=f"{dilution_range[count]}x")

axarr.set_xlabel("Time (sec)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("$f_{mol}$ Prx decamers",fontsize=mpl_ylabel_fontsize)
axarr.set_ylim(0,1)

axarr.legend(title="Dilution")
f.tight_layout()
f.savefig(os.path.join(fig_dir,"dil_fmol_Prx_decamer_vs_time_1um.pdf"),dpi= mpl_dpi)

### collated dilution assays (figure 8a, 8b)

In [ ]:
# Plot results
dil_idx = 4

print(f"Dilution of: {dilution_range[dil_idx]}x")

cols = 1
rows = 1

f, axarr = plt.subplots(rows, cols)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(result_dil_1um_Prx[dil_idx][0],result_dil_1um_Prx[dil_idx][1],label=f"1")
axarr.plot(result_dil_10um_Prx[dil_idx][0],result_dil_10um_Prx[dil_idx][1],label=f"10")
axarr.plot(result_dil_100um_Prx[dil_idx][0],result_dil_100um_Prx[dil_idx][1],label=f"100")
axarr.plot(result_dil_450um_Prx[dil_idx][0],result_dil_450um_Prx[dil_idx][1],label=f"450")

axarr.set_xlabel("Time (sec)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("$f_{mol}$ Prx decamers",fontsize=mpl_ylabel_fontsize)
axarr.set_ylim(0,1)

axarr.legend(title=f"Prx ({mu}M)")
f.tight_layout()
f.savefig(os.path.join(fig_dir,"dil_fmol_Prx_decamer_vs_time_50x.pdf"),dpi= mpl_dpi)

In [ ]:
# Plot results
dil_idx = 3

print(f"Dilution of: {dilution_range[dil_idx]}x")

cols = 1
rows = 1

f, axarr = plt.subplots(rows, cols)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(result_dil_1um_Prx[dil_idx][0],result_dil_1um_Prx[dil_idx][1],label=f"1")
axarr.plot(result_dil_10um_Prx[dil_idx][0],result_dil_10um_Prx[dil_idx][1],label=f"10")
axarr.plot(result_dil_100um_Prx[dil_idx][0],result_dil_100um_Prx[dil_idx][1],label=f"100")
axarr.plot(result_dil_450um_Prx[dil_idx][0],result_dil_450um_Prx[dil_idx][1],label=f"450")

axarr.set_xlabel("Time (sec)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("$f_{mol}$ Prx decamers",fontsize=mpl_ylabel_fontsize)
axarr.set_ylim(0,1)

axarr.legend(title=f"Prx ({mu}M)")
f.tight_layout()
f.savefig(os.path.join(fig_dir,"dil_fmol_Prx_decamer_vs_time_10x.pdf"),dpi= mpl_dpi)